In [14]:
%load_ext autoreload
%autoreload 2

import os
import sys
from glob import glob

import numpy as np
import pandas as pd
idx = pd.IndexSlice
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator

import copy

import geopandas as gpd

from sklearn.metrics import mean_absolute_error, mean_squared_error

from sklearn.linear_model import LinearRegression
from sklearn.neural_network import MLPRegressor

import gpflow
import tensorflow as tf
import sys
code_dir = '/cluster/home/kheuto01/code/zero-inflated-gp/'
sys.path.append(code_dir)
from math import radians, cos, sin, asin, sqrt
from onoffgpf import OnOffSVGP, OnOffLikelihood

import pickle

from math import radians, cos, sin, asin, sqrt

import copy
import sys

import geopandas as gpd
import numpy as np

import gpflow


code_dir = '/cluster/home/kheuto01/code/zero-inflated-gp/'
sys.path.append(code_dir)

code_dir = '/cluster/home/kheuto01/code/opioid-overdose-models/'
sys.path.append(code_dir)

from onoffgpf import OnOffSVGP, OnOffSVGPPoiMC, OnOffLikelihood
gpflow.config.default_float()


from zinf_gp.metrics import normcdf, fixed_top_X

import numpy as np 
import matplotlib.pyplot as plt 
import pandas as pd 
import contextily as ctx 
import geopandas as gpd 
import os 
from mpl_toolkits.axes_grid1 import make_axes_locatable


import matplotlib
matplotlib.style.use('default')
font = {
        'size'   : 22}

matplotlib.rc('font', **font)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:

data_dir='/cluster/tufts/hugheslab/datasets/NSF_OD/results_20220606_update/'

log_dir='/cluster/tufts/hugheslab/kheuto01/opioid/logs/fix_two/'

run_template = '{time}_{loc}_{model}_{start_year}_{cov}_{num_inducing}_{lr}'

In [3]:
time = 'annual'
loc = 'zcta'
model = 'normal'
start_year = 2000
cov = '-auto'
num_inducing = '400'
lr = 0.05

In [4]:
test_years = 2
geography_col='geoid'
timestep_col='timestep'
outcome_col='deaths'

# test y always comes from quarterly tract
y_timesteps_per_year = 4
file_name = f'clean_quarter_zcta'
data_path = os.path.join(data_dir, file_name)

x_idx_cols = [geography_col, 'lat','lon', timestep_col,
      'theme_1_pc', 'theme_2_pc', 'theme_3_pc', 'theme_4_pc',
      'svi_pctile',
      'neighbor_t', 'self_t-1']
y_idx_cols = [geography_col, timestep_col, outcome_col]
features_only = ['lat','lon', timestep_col,
                 'theme_1_pc', 'theme_2_pc', 'theme_3_pc', 'theme_4_pc',
                 'svi_pctile',
                 'neighbor_t', 'self_t-1']

data_gdf = gpd.read_file(data_path)

last_train_year = 2018
train_x_qtr = data_gdf[(data_gdf['year'] <= last_train_year)][x_idx_cols]

train_y_qtr = data_gdf[(data_gdf['year'] <= last_train_year)][y_idx_cols]

all_y_qtr = data_gdf[y_idx_cols]
test_x_qtr = data_gdf[(data_gdf['year'] > last_train_year) &
                  (data_gdf['year'] <= last_train_year+test_years)][x_idx_cols]

test_y = data_gdf[(data_gdf['year'] > last_train_year) &
                  (data_gdf['year'] <= last_train_year+test_years)][y_idx_cols]
starting_y_timestep = int(test_y[timestep_col].min())

sorted_y_timesteps = test_y[timestep_col].unique()
sorted_y_timesteps.sort()

x_timesteps_per_year = {'quarter':4, 'semi':2,'annual':1}[time]

file_name = f'clean_{time}_{loc}'
data_path = os.path.join(data_dir, file_name)

data_gdf = gpd.read_file(data_path)

train_x = data_gdf[(data_gdf['year'] <= last_train_year) &
                   (data_gdf['year'] >= start_year)][x_idx_cols]
train_y = data_gdf[(data_gdf['year'] <= last_train_year) &
                   (data_gdf['year'] >= start_year)][y_idx_cols]

test_x = data_gdf[(data_gdf['year'] > last_train_year) &
                  (data_gdf['year'] <= last_train_year+test_years)][x_idx_cols]
test_y_yr = data_gdf[(data_gdf['year'] > last_train_year) &
                  (data_gdf['year'] <= last_train_year+test_years)][y_idx_cols]
starting_x_timestep = int(test_x[timestep_col].min())

test_timesteps_per_year = max(y_timesteps_per_year, x_timesteps_per_year)
test_timesteps = test_timesteps_per_year*test_years

x_repeats = int(test_timesteps_per_year/x_timesteps_per_year)
y_repeats = int(test_timesteps_per_year/y_timesteps_per_year)

sorted_x_timesteps = test_x[timestep_col].unique()
sorted_x_timesteps.sort()

x_timesteps = [timestep  for timestep in sorted_x_timesteps for _ in range(x_repeats)]

y_timesteps = [timestep  for timestep in sorted_y_timesteps for _ in range(y_repeats)]



In [5]:
this_run = run_template.format(time=time,loc=loc,model=model,start_year=start_year,
                                                           cov=cov,
                                                           num_inducing=num_inducing,lr=lr)
with open(os.path.join(log_dir,this_run,'model.mod'),'rb') as f:
        predictor = pickle.load(f)
with open(os.path.join(log_dir,this_run,'stats.csv'),'rb') as f:
    stats = pd.read_csv(f)

2023-03-02 10:07:01.535450: E tensorflow/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2023-03-02 10:07:01.535501: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: p1cmp078.pax.tufts.edu
2023-03-02 10:07:01.535510: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: p1cmp078.pax.tufts.edu
2023-03-02 10:07:01.535689: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:200] libcuda reported version is: 510.47.3
2023-03-02 10:07:01.535722: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:204] kernel reported version is: 510.47.3
2023-03-02 10:07:01.535729: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:310] kernel version seems to match DSO: 510.47.3
2023-03-02 10:07:01.536242: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the fo

In [6]:
new_tract_shapefile = os.path.join(os.path.dirname(os.path.dirname(data_dir)),'shapefiles','new_trimmed_tract',
                                   'cb_2020_tract')
town_shapefile = os.path.join(os.path.dirname(os.path.dirname(data_dir)),'shapefiles','MA_2020_Towns',
                              'CENSUS2020TOWNS_POLY.shp')
mass_gdf = gpd.read_file(new_tract_shapefile)
town_gdf = gpd.read_file(town_shapefile)


In [11]:
last_year

geoid
01001     6.0
01002     2.0
01003     0.0
01005     3.0
01007     5.0
         ... 
02777     4.0
02779     1.0
02780    32.0
02790     8.0
02791     0.0
Name: deaths, Length: 537, dtype: float64

In [34]:
np.sqrt(mean_squared_error(y_2019, last_5))

2.3589916729869143

In [28]:
mean_absolute_error(y_2019, last_5)

1.3869646182495343

In [72]:
y_2019 = test_y_yr[test_y_yr[timestep_col]==19].groupby('geoid').sum()['deaths']
last_year = train_y[train_y[timestep_col] ==18].set_index('geoid')['deaths']
last_5 = train_y[train_y[timestep_col].isin([18,17,16,15,14])].groupby('geoid').mean()['deaths']

last_3 = train_y[train_y[timestep_col].isin([18,17,16])].groupby('geoid').mean()['deaths']

last_2 = train_y[train_y[timestep_col].isin([18,17])].groupby('geoid').mean()['deaths']
fixed_top_X(y_2019,last_year,100)[-1]
ly_labels = pd.qcut(last_year,5, duplicates='drop', labels=False)
y_2019_labels =pd.qcut(y_2019,5, duplicates='drop', labels=False)
accuracy_score(ly_labels, y_2019_labels)
np.sqrt(mean_squared_error(y_2019, last_5))
mean_absolute_error(y_2019, last_5)

1.3869646182495343

In [23]:
last_5

geoid
01001     3.0
01002     1.8
01003     0.0
01005     1.0
01007     3.0
         ... 
02777     2.8
02779     2.2
02780    22.6
02790     4.6
02791     0.0
Name: deaths, Length: 537, dtype: float64

In [14]:
bprs = []
for t in [76,77,78,79]:
    y_2019 = test_y[test_y[timestep_col].isin([t])].groupby('geoid').sum()['deaths']
    last_year = train_y[train_y[timestep_col] ==18].set_index('geoid')['deaths']
    bprs.append(fixed_top_X(y_2019,last_year,100)[-1])

In [ ]:
trials_2019 = []
for _ in range(100):
    bprs = []
    for t in [76,77,78,79]:
        y_2019 = test_y[test_y[timestep_col].isin([t])].groupby('geoid').sum()['deaths']
        last_year = train_y[train_y[timestep_col] ==18].set_index('geoid')['deaths']
        bprs.append(fixed_top_X(y_2019,last_year,100)[-1])
    trials_2019.append(np.mean(bprs))

In [ ]:
bprs

In [16]:
bprs = []
for t in [76,77,78,79]:
    y_2019 = test_y[test_y[timestep_col].isin([t])].groupby('geoid').sum()['deaths']
    last_2019 = all_y_qtr[all_y_qtr[timestep_col].isin([t-1])].groupby('geoid').sum()['deaths']
    bprs.append(fixed_top_X(y_2019,last_2019,100)[-1])

In [17]:
bprs

[0.2577134502923976,
 0.24423728813559323,
 0.26881283422459895,
 0.2580765027322404]

In [ ]:
trials_2020 = []
for _ in range(100):
    bprs = []
    for t in [80,81,82,83]:
        y_2020 = test_y[test_y[timestep_col].isin([t])].groupby('geoid').sum()['deaths']
        last_year = test_y_yr[test_y_yr[timestep_col] ==19].set_index('geoid')['deaths']
        bprs.append(fixed_top_X(y_2020,last_year,100)[-1])
    trials_2020.append(np.mean(bprs))

In [23]:
np.mean(bprs)

0.3304623841419298

In [26]:
np.mean(bprs)

0.32982486060778476

In [33]:
trials_2020

[0.33028024016555974,
 0.32927093377046995,
 0.3290402419747322,
 0.32883976904707013,
 0.33005354203887655,
 0.3311347665588404,
 0.33011345283137394,
 0.3289451836110406,
 0.32984003398388384,
 0.32940655812709074,
 0.33002540349438136,
 0.3290916083170681,
 0.32997528500075257,
 0.3286589850521387,
 0.33127410097646226,
 0.3306732001143512,
 0.32953695065085825,
 0.32919656452548834,
 0.32948533510285094,
 0.3298039284551785,
 0.33038152380124847,
 0.3292995541325753,
 0.3293715745270849,
 0.33006215140606976,
 0.33102659484340247,
 0.33061877255532746,
 0.3302294290298825,
 0.3297505191621146,
 0.3296627451413712,
 0.3296587566417858,
 0.33009203950506316,
 0.32939395215884487,
 0.330244719566442,
 0.3298634498710277,
 0.32958031596500226,
 0.330642387364779,
 0.3299828193431177,
 0.33015631440676674,
 0.3292406228218574,
 0.3311993603173712,
 0.3300954645649067,
 0.32989225298085406,
 0.32968013398024876,
 0.32977838737759235,
 0.3299167279915955,
 0.33069858614430964,
 0.32974770

In [34]:
trials_2019

[0.3538270953995545,
 0.3529761823222952,
 0.3539537588440004,
 0.3538922250100693,
 0.3540362086147237,
 0.3548128472127999,
 0.3538805460345678,
 0.3535569375314153,
 0.3533425327789476,
 0.3545632293231078,
 0.35382395054532756,
 0.35409679804634864,
 0.3542748083956878,
 0.3540713044692975,
 0.3544942711845812,
 0.3542843355050022,
 0.354138297003604,
 0.3529507884431001,
 0.3547938531843484,
 0.35395589680961903,
 0.3534143126481147,
 0.35448016531844406,
 0.353371423149608,
 0.3539314127952813,
 0.35377428004158273,
 0.3545055047553759,
 0.35398025290136503,
 0.35467203880510356,
 0.35453354777137636,
 0.35371450951613487,
 0.3539941106274782,
 0.35438988832866924,
 0.3540977845913371,
 0.3547581637327148,
 0.3547325872019208,
 0.3544418801621473,
 0.3539914456176533,
 0.354507480890922,
 0.3547029452034863,
 0.3544694235636142,
 0.3548542563518341,
 0.35375233348811425,
 0.35457036125741354,
 0.3540574191093152,
 0.3550005753453682,
 0.35360497287518833,
 0.35444886197943937,
 0

In [52]:
y_2019 = train_y[train_y[timestep_col]==18].groupby('geoid').sum()['deaths']
last_year = train_y[train_y[timestep_col] ==17].set_index('geoid')['deaths']
last_5 = train_y[train_y[timestep_col].isin([13,17,16,15,14])].groupby('geoid').mean()['deaths']
np.sqrt(mean_squared_error(y_2019, last_5))
fixed_top_X(y_2019, last_5, 100)

(78.0, 0.9366883116883117, 83.306, 0.9323814935064935)

In [39]:
y_2019 = train_y[train_y[timestep_col]==17].groupby('geoid').sum()['deaths']
last_year = train_y[train_y[timestep_col] ==17].set_index('geoid')['deaths']
last_5 = train_y[train_y[timestep_col].isin([13,12,16,15,14])].groupby('geoid').mean()['deaths']
np.sqrt(mean_squared_error(y_2019, last_5))

2.4617294188930625

In [41]:
y_2019 = train_y[train_y[timestep_col]==16].groupby('geoid').sum()['deaths']
last_year = train_y[train_y[timestep_col] ==15].set_index('geoid')['deaths']
last_5 = train_y[train_y[timestep_col].isin([13,12,11,15,14])].groupby('geoid').mean()['deaths']
np.sqrt(mean_squared_error(y_2019, last_year))

2.626324264707083

In [46]:
(last_year==0).sum()

221

In [56]:
np.quantile(last_year,0.4)

0.4000000000000057

In [67]:
y_2019_labels.unique()

array([3, 1, 0, 2])

In [68]:
from sklearn.metrics import accuracy_score

0.6126629422718808